In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import copy

In [2]:
pcd = o3d.io.read_point_cloud("C:/Program Files (x86)/Steam/steamapps/common/SteamCMD/csgo-ds/csgo/test.xyz", format='xyzn')
print(pcd)
#o3d.visualization.draw_geometries([pcd])

PointCloud with 1206994 points.


In [3]:
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10)
print(mesh)

run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 1206994 / 1124086
[Open3D DEBUG] #   Got kernel density: 0.521 (s), 504.168 (MB) / 504.168 (MB) / 504 (MB)
[Open3D DEBUG] #     Got normal field: 2.643 (s), 1018.32 (MB) / 1018.32 (MB) / 1018 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.004413e-06 / 3.626308e+00
[Open3D DEBUG] #       Finalized tree: 2.60500 (s), 1362.31 (MB) / 1362.31 (MB) / 1566 (MB)
[Open3D DEBUG] #  Set FEM constraints: 8.165 (s), 1143.81 (MB) / 1362.31 (MB) / 1566 (MB)
[Open3D DEBUG] #Set point constraints: 1.15 (s), 1039.55 (MB) / 1362.31 (MB) / 1566 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 13961942 / 15531504 / 425001
[Open3D DEBUG] Memory Usage: 1039.566 MB
[Open3D DEBUG] # Linear system solved: 13.281 (s), 1552.2 (MB) / 1552.2 (MB) / 1566 (MB)
[Open3D DEBUG] Got average: 0.324 (s), 1008.48 (MB) / 1552.2 (MB) / 1566 (MB)
[Open3D DEBUG] Iso-Value: 5.021038e-01 = 6.060363e+05 / 1.206994e+06
[Open3D DEBUG] #      

In [4]:
mesh.compute_vertex_normals()
mesh.paint_uniform_color(np.array([0.9, 0.8, 0.8]))
o3d.visualization.draw_geometries([mesh])

In [5]:
z_min = -240
z_max = 280

zs = np.asarray(mesh.vertices)[:, 2]
mask = (zs > z_max) | (zs < z_min)

mesh.remove_vertices_by_mask(mask)

In [6]:
print("Simplify mesh. Might take a while.")
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    mesh = mesh.simplify_quadric_decimation(500000)

Simplify mesh


In [7]:
print("Cluster connected triangles")
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    triangle_clusters, cluster_n_triangles, cluster_area = mesh.cluster_connected_triangles()
triangle_clusters = np.asarray(triangle_clusters)
cluster_n_triangles = np.asarray(cluster_n_triangles)
cluster_area = np.asarray(cluster_area)

print("Remove small clusters")
triangles_to_remove = cluster_n_triangles[triangle_clusters] < 200
mesh.remove_triangles_by_mask(triangles_to_remove)

Cluster connected triangles
[Open3D DEBUG] [ClusterConnectedTriangles] Compute triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done computing triangle adjacency
[Open3D DEBUG] [ClusterConnectedTriangles] Done clustering, #clusters=97
Remove small clusters


In [8]:
print(mesh)
o3d.visualization.draw_geometries([mesh])

TriangleMesh with 252963 points and 498329 triangles.
